In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GCS_OUTPUT_PATH = "gs://doc_parser12345/"
# PROCESSOR_NAME = "https://us-documentai.googleapis.com/v1/projects/712357922075/locations/us/processors/5d98407fb8df954c:process"
PROCESSOR_NAME = "projects/712357922075/locations/us/processors/5d98407fb8df954c"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './docextraction-440803-2193a2f4556f.json'


In [3]:
from langchain_core.document_loaders.blob_loaders import Blob
from langchain_google_community import DocAIParser

In [4]:
parser = DocAIParser(
    location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
)

In [5]:
blob = Blob(
    path="gs://doc_parser12345/document-001-115484.out.000.pdf"
)

In [6]:
docs = list(parser.lazy_parse(blob))
print(len(docs))

1


In [7]:
print(len(docs))

1


In [8]:
docs

[Document(metadata={'page': 1, 'source': 'gs://doc_parser12345/document-001-115484.out.000.pdf'}, page_content='SHARP\nGL1F20\nGL1F20\nFeatures\n1. IrDA1.0 compatible infrared emitting diode\n(Transmission rate: 2.4 to 115.2kbps)\n2. Built-in infrared emitting diode circuit\n3. Recommended use in combination with detector (IS1U20)\nInfrared Communication\n(IrDA1.0 Compatible)\nInfraredEmitting Diode\n| Outline Dimensions\nMAX. 0.6\n5.6\n(Unit: mm)\nR 1.4\nTransparent resin\nDetector center\nApplications\n1. Personal computers\n2. Portable information terminal equipment\n3. Printers\n4. Word processors\nIrDA Abbreviation of the Infrared Data Association established\nfor standardization of infrared communication specifications\n3-0.45\n1.27\nAbsolute Maximum Ratings\nParameter\nForward current\n*1 Peak forward current\nOperating temperature\nStorage temperature\n#2\nSoldering temperature\nA\n17.9\n1.0\n1.27\nMIN 0.3\n(Ta=25°C)\nSymbol Rating\nUnit\nIF\n50\nmA\nTerminal configuration\n1 B

In [9]:
type(docs[0])

langchain_core.documents.base.Document

In [10]:
# Assuming 'docs' is a list of Document objects with 'page_content' as a text attribute

# Extract and clean the text from page_content
clean_content = docs[0].page_content.replace('\n', ' ')

# Store the cleaned content in a variable
clean_content


'SHARP GL1F20 GL1F20 Features 1. IrDA1.0 compatible infrared emitting diode (Transmission rate: 2.4 to 115.2kbps) 2. Built-in infrared emitting diode circuit 3. Recommended use in combination with detector (IS1U20) Infrared Communication (IrDA1.0 Compatible) InfraredEmitting Diode | Outline Dimensions MAX. 0.6 5.6 (Unit: mm) R 1.4 Transparent resin Detector center Applications 1. Personal computers 2. Portable information terminal equipment 3. Printers 4. Word processors IrDA Abbreviation of the Infrared Data Association established for standardization of infrared communication specifications 3-0.45 1.27 Absolute Maximum Ratings Parameter Forward current *1 Peak forward current Operating temperature Storage temperature #2 Soldering temperature A 17.9 1.0 1.27 MIN 0.3 (Ta=25°C) Symbol Rating Unit IF 50 mA Terminal configuration 1 Base IFM 400 mA 12: Emitter Topr 10 to 70 °C ③3 Anode Iste 20 to + 85 °C Tsol 260 °C *1 Pulse width 78.1 u s. Duty ratio=3/16 *2 For MAX. 3 seconds at the posi

# Stuff: summarize in a single LLM call

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


c:\Code\Github\document_classifier\classifier\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me what this document is all about?, categorise it as a type of document:\\n\\n{context}")]
)


# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

This document is a **datasheet** for the **SHARP GL1F20 infrared emitting diode**. It provides technical specifications and information about the device, including:

* **Features:**  Describes the key capabilities of the diode, such as its compatibility with IrDA 1.0 standard, transmission rate, and built-in circuit.
* **Outline Dimensions:**  Illustrates the physical dimensions of the diode.
* **Applications:**  Lists typical applications for the diode, including personal computers, portable equipment, printers, and word processors.
* **Absolute Maximum Ratings:** Specifies the maximum allowable values for parameters such as forward current, operating temperature, and storage temperature.
* **Electro-optical Characteristics:**  Provides detailed electrical and optical specifications, including driving voltage, peak emission wavelength, radiant intensity, light pulse width, and input current.

**Overall, the document aims to provide engineers and designers with comprehensive informatio

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("user", "Extract all tables from it and give output as markdown:\\n\\n{context}")]
)


# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

## SHARP GL1F20 Infrared Emitting Diode

### Features

1. IrDA1.0 compatible infrared emitting diode (Transmission rate: 2.4 to 115.2kbps)
2. Built-in infrared emitting diode circuit
3. Recommended use in combination with detector (IS1U20)

### Infrared Communication (IrDA1.0 Compatible)

#### Infrared Emitting Diode

| Outline Dimensions | |
|---|---|
| **MAX. 0.6** | (Unit: mm) |
| **5.6** | |
| **R 1.4** | |
| **Transparent resin** | |
| **Detector center** | |
| **3-0.45** | |
| **1.27** | |
| **MIN 0.3** | |
| **17.9** | |
| **1.0** | |
| **1.27** | |

### Applications

1. Personal computers
2. Portable information terminal equipment
3. Printers
4. Word processors

### IrDA Abbreviation

Infrared Data Association established for standardization of infrared communication specifications.

### Absolute Maximum Ratings

| Parameter | Symbol | Rating | Unit |
|---|---|---|---|
| Forward current | IF | 50 | mA |
| Peak forward current | IFM | 400 | mA |
| Operating temperature | Topr | 

## SHARP GL1F20 Infrared Emitting Diode

### Features

1. IrDA1.0 compatible infrared emitting diode (Transmission rate: 2.4 to 115.2kbps)
2. Built-in infrared emitting diode circuit
3. Recommended use in combination with detector (IS1U20)

### Infrared Communication (IrDA1.0 Compatible)

#### Infrared Emitting Diode

| Outline Dimensions (Unit: mm) | | |
|---|---|---|
| MAX. 0.6 | R 1.4 | 5.6 |
| 3-0.45 | 1.27 | 1.0 |
| MIN 0.3 | 1.27 | 17.9 |
| | Transparent resin |  |
| | Detector center |  |

### Applications

1. Personal computers
2. Portable information terminal equipment
3. Printers
...
*3 Direction of mechanical axis of the lens portion of the element: =0°

**Note:** In the absence of confirmation by device specification sheets, SHARP takes no responsibility for any defects that occur in equipment using any of SHARP's devices, shown in catalogs, data books, etc. Contact SHARP in order to obtain the latest version of the device specification sheets before using any SHARP's device. 

## SHARP GL1F20 Infrared Emitting Diode

### Features

1. IrDA1.0 compatible infrared emitting diode (Transmission rate: 2.4 to 115.2kbps)
2. Built-in infrared emitting diode circuit
3. Recommended use in combination with detector (IS1U20)

### Infrared Communication (IrDA1.0 Compatible)

#### Infrared Emitting Diode

| Outline Dimensions | |
|---|---|
| **MAX. 0.6** | (Unit: mm) |
| **5.6** | |
| **R 1.4** | |
| **Transparent resin** | |
| **Detector center** | |
| **3-0.45** | |
| **1.27** | |
| **MIN 0.3** | |
| **17.9** | |
| **1.0** | |
| **1.27** | |

### Applications

1. Personal computers
2. Portable information terminal equipment
3. Printers
4. Word processors

### IrDA Abbreviation

Infrared Data Association established for standardization of infrared communication specifications.

### Absolute Maximum Ratings

| Parameter | Symbol | Rating | Unit |
|---|---|---|---|
| Forward current | IF | 50 | mA |
| Peak forward current | IFM | 400 | mA |
| Operating temperature | Topr | 10 to 70 | °C |
| Storage temperature | Iste | -20 to + 85 | °C |
| Soldering temperature | Tsol | 260 | °C |

*1 Pulse width 78.1 u s. Duty ratio=3/16
*2 For MAX. 3 seconds at the position of 2 mm from the resin edge

### Electro-optical Characteristics (Ta=25 °C)

| Parameter | Symbol | Conditions | MIN. | TYP. | MAX. | Unit |
|---|---|---|---|---|---|---|
| Driving voltage | Vcc |  | 4.75 | 5.25 | V |
| High level input voltage | VIII |  | 4.5 | Voc | V |
| Low level input voltage | VII. |  | 0.4 | V |
| Peak emission wavelength | Ap | IF = 20mA | 850 | 870 | 900 | nm |
| Radiant intensity | JE | Vcc 5V,RL= 7.592 | 40 | 350 | mW/sr |
| Light pulse width | tw | Vin = 4.5V | 1.41 | 1.6 | 2.71 | μs |
| Light rise time | tr | twin 1.63 us,Duty ratio : 3/16 | 0.23 | 0.6 | μs |
| Light fall time | lf | <=15°,*3 | 0.17 | 0.6 | μs |
| Input current | IIII | Vin = 4.5V | 1.0 | 3.0 | mA |
| Half intensity wavelength | Δλ | Ir = 20mA | 40 | nm |
| Half intensity angle | ΔΕ | IF = 20mA | - | ± 20 |  |

*3 Direction of mechanical axis of the lens portion of the element: =0°

**Disclaimer:** In the absence of confirmation by device specification sheets, SHARP takes no responsibility for any defects that occur in equipment using any of SHARP's devices, shown in catalogs, data books, etc. Contact SHARP in order to obtain the latest version of the device specification sheets before using any SHARP's device. 